# Práctico 9: Redes atencionales

La Tarea de Redes Atencionales (ANT, por sus siglas en inglés) es una tarea experimental utilizada para evaluar el funcionamiento de las redes atencionales en el cerebro. Este paradigma, desarrollado por [Fan et al. (2002)](https://doi.org/10.1162/089892902317361886), permite analizar tres tipos funcionales de la atención:

- Alerta: la habilidad para alcanzar y mantener un estado de alta sensibilidad a estímulos.
- Orientación: la capacidad de dirigir la atención hacia la ubicación espacial del estímulo relevante.
- Control ejecutivo: el proceso de gestionar y resolver conflictos entre respuestas o estímulos en competencia.

En la ANT, los participantes responden a un estímulo visual central (por ejemplo, una flecha que apunta a la izquierda o derecha) tras recibir pistas que pueden ser útiles para predecir la aparición o la ubicación del estímulo. La velocidad y precisión de las respuestas en diferentes condiciones permiten evaluar cada componente de la atención.

Este tipo de tarea es muy útil en estudios de neurociencia cognitiva, ya que mide las diferencias individuales en habilidades atencionales y facilita el análisis de cómo diversos factores, como el envejecimiento o ciertas condiciones clínicas, afectan la atención.

## Configuración

Ejecutá las siguientes celdas para configurar el entorno del notebook.

In [ ]:
import requests, io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pistas = ["no", "centro", "doble", "espacial"]
congruencias = ["congruente", "incongruente", "neutral"]

### Funciones utilitarias

In [ ]:
def descargar_datos(nombre_de_archivo):
    url = "https://gitlab.pavlovia.org/demos/attention_network_task/-/raw/master/" + nombre_de_archivo + "?inline=false"

    response = requests.get(url)
    response.raise_for_status()
    
    df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    
    return preprocesar_datos(df)

def preprocesar_datos(df):
    # Nos quedamos con las columnas 17, 18, 22, 27 y 28
    # Columna 17: Acierto, 0 si cometió un error, 1 si acertó.
    # Columna 18: Tiempo de respuesta, en ms.
    # Columna 22: Repetición.
    datos = df.values[:, [17, 18, 22, 27, 28]].T
    datos[3, np.equal(datos[3], 'stim/blank.png')] = "no"
    datos[3, np.equal(datos[3], 'stim/both.png')] = "doble"
    datos[3, np.equal(datos[3], 'stim/centre.png')] = "centro"
    datos[3, np.isin(datos[3], ['stim/lower.png', 'stim/upper.png'])] = "espacial"
    datos[4, np.isin(datos[4], ['stim/congRight.png', 'stim/congLeft.png'])] = "congruente"
    datos[4, np.isin(datos[4], ['stim/incongRight.png', 'stim/incongLeft.png'])] = "incongruente"
    datos[4, np.isin(datos[4], ['stim/neutralRight.png', 'stim/neutralLeft.png'])] = "neutral"
    
    # Eliminamos algunas filas que no tienen información
    datos = datos[:,~np.isnan(np.array(datos[1], dtype=np.float64))]
    
    # Y las de practica
    datos = datos[:,~np.equal(np.array(datos[2], dtype=np.float64), 1)]
    
    # Borramos la columna que indica si es entrenamiento, porque no la usaremos más
    datos = np.delete(datos, 2, 0)

    return datos

def aciertos_por_pista(pista):
    """Devuelve los aciertos para una pista dada, separados por tipo de congruencia.

    Args:
        pista (str): Nombre de la pista. Uno de {"no", "centro", "doble", "espacial"}.
    """
    return np.array([datos[0,(datos[2] == pista) & (datos[3] == congruencia)] for congruencia in congruencias], dtype=np.float64)
    
def tiempos_de_respuesta_por_pista(pista):
    """Devuelve los tiempos de respuesta para una pista dada, separados por tipo de congruencia.
    
    Args:
        pista (str): Nombre de la pista. Uno de {"no", "centro", "doble", "espacial"}.
    """
    return np.array([datos[1,(datos[2] == pista) & (datos[3] == congruencia)] for congruencia in congruencias], dtype=np.float64)

def tiempos_de_respuesta_por_congruencia(congruencia):
    """Devuelve los tiempos de respuesta para una congruencia dada, separados por tipo de pista.

    Args:
        congruencia (str): Tipo de congruencia. Uno de {"congruente", "incongruente", "neutral"}.
    """
    return np.array([datos[1,(datos[2] == pista) & (datos[3] == congruencia)] for pista in pistas], dtype=np.float64)

## Ejecución de la tarea

Antes que nada, queremos que corras la tarea. No sólo para que comprendas mejor de que se trata, sino porque también queremos que trabajes con tus propios datos.

Asegurate de seguir los siguientes pasos al pie de la letra:

1. Accedé a una versión de esta tarea que se encuentra disponible aquí: <https://run.pavlovia.org/demos/attention_network_task>.

2. Para luego poder descargar los datos, es **muy importante** que antes de comenzar la tarea te anotes el número de participante que se te asignó.

![Tarea](https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico9_Tarea.png)

3. Leé las instrucciones y realizá la tarea. Tratá de responder **todos los ensayos** ya que para simplificar los análisis luego asumiremos que todos los ensayos tuvieron alguna respuesta.

## Exploración de los datos

Para descargar los datos, es necesario encontrar primero el nombre exacto del archivo que generó la tarea. Asegurate de seguir los siguientes pasos:

1. Buscá el archivo con sus datos a partir del número de participante que se te asignó [en esta página](https://gitlab.pavlovia.org/demos/attention_network_task/-/find_file/master). Debe decir `XXXX_attention_network_task_YYYY`, donde `XXXX` es tu número de participante, mientras que `YYYY` es la hora y la fecha.

![Buscar](https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico9_Buscar.png)

2. Hacé clic en el archivo que encontraste, y luego en el botón para copiar la ruta al archivo.

![Copiar](https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico9_Copiar.png)

3. Reemplaza el valor de `nombre_de_archivo` en el código a continuación, con el nombre de archivo que encontraste.

In [ ]:
datos = descargar_datos(nombre_de_archivo="data/890547_attention_network_task_2025-11-12_16h52.00.593.csv")
display(datos)

Tratemos de poner un poco de orden. ¿Que dimensiones tienen los datos?

In [ ]:
print("Forma de los datos", datos.shape)

Los datos estan guardados en una matriz de $4 \times n$, donde las primera dimension contiene cuatro filas:

0. Acierto, 0 si cometió un error, 1 si acertó.
1. Tiempo de respuesta, en milisegundos.
2. _Cue_
3. _Target_

Mientras que la segunda dimension contiene tantas columnas como ensayos hayas completado.

Usá la siguiente celda para explorar los datos. Probá imprimir las cuatro filas que mencionamos.

In [ ]:
print("Fila:", datos[0])

## Análisis preliminares

Les dejamos tres funciones definidas para poder extraer datos en forma estructurada: `aciertos_por_pista`, `tiempos_de_respuesta_por_congruencia` y `tiempos_de_respuesta_por_pista`.

En esta sección, deberás probar que salidas devuelven para hacerte una idea de qué operación realizan y que estructuras tienen sus salidas.

### `aciertos_por_pista`

Ejecutá la siguiente celda para imprimir información sobre `aciertos_por_pista`.

In [ ]:
help(aciertos_por_pista)

Usá la siguiente celda para llamar la función. Probá con diferentes pistas.

In [ ]:
print(aciertos_por_pista("espacial"))

### `tiempos_de_respuesta_por_congruencia`

Ejecutá la siguiente celda para imprimir información sobre `tiempos_de_respuesta_por_congruencia`.

In [ ]:
help(tiempos_de_respuesta_por_congruencia)

Usá la siguiente celda para llamar la función. Probá con diferentes congruencias.

In [ ]:
print(tiempos_de_respuesta_por_congruencia("neutral"))

### `tiempos_de_respuesta_por_pista`

Ejecutá la siguiente celda para imprimir información sobre `tiempos_de_respuesta_por_pista`.

In [ ]:
help(tiempos_de_respuesta_por_pista)

Usá la siguiente celda para llamar la función. Probá con diferentes pistas.

In [ ]:
print(tiempos_de_respuesta_por_pista("doble"))

### Visualización de los tiempos de reacción

In [ ]:
plt.figure(figsize=[8,6])
plt.errorbar(congruencias, np.mean(tiempos_de_respuesta_por_pista("no"), axis=1), yerr=np.std(tiempos_de_respuesta_por_pista("no"), axis=1), fmt="-o")
plt.errorbar(congruencias, np.mean(tiempos_de_respuesta_por_pista("centro"), axis=1), yerr=np.std(tiempos_de_respuesta_por_pista("centro"), axis=1), fmt="--s")
plt.errorbar(congruencias, np.mean(tiempos_de_respuesta_por_pista("doble"), axis=1), yerr=np.std(tiempos_de_respuesta_por_pista("doble"), axis=1), fmt="-^")
plt.errorbar(congruencias, np.mean(tiempos_de_respuesta_por_pista("espacial"), axis=1), yerr=np.std(tiempos_de_respuesta_por_pista("espacial"), axis=1), fmt=":o")
plt.legend(pistas)
plt.show()

### Visualización del porcentaje de errores

In [ ]:
plt.figure(figsize=[6,6])
plt.errorbar(congruencias, 1-np.mean(aciertos_por_pista("no"), axis=1), yerr=np.std(aciertos_por_pista("no"), axis=1), fmt="-o")
plt.errorbar(congruencias, 1-np.mean(aciertos_por_pista("centro"), axis=1), yerr=np.std(aciertos_por_pista("centro"), axis=1), fmt="--s")
plt.errorbar(congruencias, 1-np.mean(aciertos_por_pista("doble"), axis=1), yerr=np.std(aciertos_por_pista("doble"), axis=1), fmt="-^")
plt.errorbar(congruencias, 1-np.mean(aciertos_por_pista("espacial"), axis=1), yerr=np.std(aciertos_por_pista("espacial"), axis=1), fmt=":o")
plt.legend(pistas)
plt.show()

## Cálculo de los efectos de las redes atencionales

Extraemos unos promedios generales de los tiempos de reacción de las condiciones, que usaremos para medir los diferentes tipos de atención

In [ ]:
print("Promedio de tiempos de reacción para NO:", np.round(np.mean(tiempos_de_respuesta_por_pista("no")), 3), "segundos")
print("Promedio de tiempos de reacción para CENTRO:", np.round(np.mean(tiempos_de_respuesta_por_pista("centro")), 3), "segundos")
print("Promedio de tiempos de reacción para DOBLE:", np.round(np.mean(tiempos_de_respuesta_por_pista("doble")), 3), "segundos")
print("Promedio de tiempos de reacción para ESPACIAL:", np.round(np.mean(tiempos_de_respuesta_por_pista("espacial")), 3), "segundos")

In [ ]:
print("Promedio de tiempos de reacción para INCONGRUENTES:", np.round(np.mean(tiempos_de_respuesta_por_congruencia("incongruente")), 3), "segundos")
print("Promedio de tiempos de reacción para CONGRUENTES:", np.round(np.mean(tiempos_de_respuesta_por_congruencia("congruente")), 3), "segundos")
print("Promedio de tiempos de reacción para NEUTRALES:", np.round(np.mean(tiempos_de_respuesta_por_congruencia("neutral")), 3), "segundos")

### Efecto de "alerta"

Es el efecto que tiene "alertar" sobre la inminencia del estímulo. Se calcula como el promedio de los TR de la condición sin cue espacial, menos el promedio de la condición con cue espacial doble. Esta última condición no dirige la atención hacia ningún lugar en particular (no aporta información acerca de dónde va a aparecer el estímulo), pero sí pone en "alerta" al sistema.

In [ ]:
efecto_de_alerta = np.mean(tiempos_de_respuesta_por_pista("no")) - np.mean(tiempos_de_respuesta_por_pista("doble"))

print("Efecto de ALERTA:", np.round(efecto_de_alerta * 1000, 1), "milisegundos")

### Efecto de "orientación"

Es el efecto de orientar la atención sobre una región particular del espacio. Se calcula como el promedio de los TR de la condición con cue central, menos el promedio de la condición con cue espacial (la que verdaderamente predice la posición del target).

In [ ]:
efecto_de_orientacion = np.mean(tiempos_de_respuesta_por_pista("centro")) - np.mean(tiempos_de_respuesta_por_pista("espacial"))

print("Efecto de ORIENTACIÓN:", np.round(efecto_de_orientacion * 1000, 1), "milisegundos")

### Efecto de "conflicto"

In [ ]:
efecto_de_conflicto = np.mean(tiempos_de_respuesta_por_congruencia("incongruente")) - np.mean(tiempos_de_respuesta_por_congruencia("congruente"))

print("Efecto de CONFLICTO:", np.round(efecto_de_conflicto * 1000, 1), "milisegundos")

## Cargar varios archivos de datos

Es hora de visualizar datos acumulados con los datos de la clase. Usá la siguiente celda para cargar los datos de tus compañeras y compañeros, para luego poder analizarlos en conjunto.

In [ ]:
# Cargar archivos csv
datos1 = descargar_datos(nombre_de_archivo="data/890547_attention_network_task_2025-11-12_16h52.00.593.csv")
datos2 = descargar_datos(nombre_de_archivo="data/395570_attention_network_task_2025-10-14_09h47.03.465.csv")

# Concatenarlos
datos = np.concat([datos1, datos2], axis=1)

print("Forma de los datos", datos.shape)

Acabamos de sobreescribir la variable `datos`. Volvé a correr las celdas a partir de la sección "Análisis preliminares" para que los análisis ahora usen estos datos acumulados.

## Referencias

Fan, J., et al (2002). Testing the Efficiency and Independence of Attentional Networks. *Journal of Cognitive Neuroscience*, 14(3), 375–388. [doi:10.1162/089892902317361886](https://doi.org/10.1162/089892902317361886)